In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
import os
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd
from sklearn.metrics import r2_score
from scipy.interpolate import interp1d
import tensorflow_datasets as tfds
from collections import Counter
import scipy.ndimage
from tensorflow.keras.callbacks import ModelCheckpoint
import keras
import gc
import random
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
import tarfile
import urllib.request
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import torch.nn.functional as F
from scipy.stats import gaussian_kde
from numba import jit
from torchvision.datasets import INaturalist
from tqdm import tqdm  # 導入進度條庫
import json


os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
torch.backends.cudnn.benchmark = True
limit = 1000

# bzq modifying
len_x_target = 20
len_y_target = 20
stride_x_target = 10
stride_y_target = 10

# mean, std proportion
alpha = 0.5

bins_size = 30  # 統計採樣數
poly_degree = bins_size - 1
window_size = 1

#target image preprocessing
angle = 0
pixels = 0

In [ ]:
# 設置設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

def adjust_learning_rate(optimizer, epoch, initial_lr):
    if epoch <= 5:
        lr = initial_lr * epoch / 5
    elif epoch > 160:
        lr = initial_lr * 0.01
    elif epoch > 75:
        lr = initial_lr * 0.1
    else:
        lr = initial_lr

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    #transforms.RandomCrop((224, 224)),  # 隨機裁剪至 224x224
    transforms.Lambda(lambda img: img.convert("RGB")),  # 確保所有圖像都是 RGB 格式
    #transforms.RandomHorizontalFlip(p=0.5),  # 機率為 50% 隨機水平翻轉
    transforms.ToTensor(),  # 將圖像轉換為 Tensor
    transforms.Normalize(mean=[0.466, 0.471, 0.380], std=[0.195, 0.194, 0.192]),  # 標準化
])

dataset = INaturalist(
    root="/home/a/bzq_on_confidence/Confidence2022/notebook/confident/grocery/iNaturalist2018/train_val2018",  # 資料集存儲的根目錄
    version="2018",          # 使用 2018 版本
    target_type="super",     # 使用超級分類
    transform=transform,     # 圖像增強
    download=False           # 如果需要下載設置為 True
    )

model = models.resnet50(weights=False)
# 獲取原始模型的輸入特徵大小
num_features = model.fc.in_features

# 修改全連接層輸出至 14 類別（對應 iNaturalist 2018 的 super category）
model.fc = torch.nn.Linear(num_features, 14)
model = model.to(device)

# 定義損失函數和優化器
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
num_epochs = 50

if os.path.exists(f"resnet50_epoch_{num_epochs}.pth"):
    # 加載保存的權重
    checkpoint_path = f"resnet50_epoch_{num_epochs}.pth"  # 你的權重檔案路徑
    model.load_state_dict(torch.load(checkpoint_path))  # 讀取權重並更新模型參數

    # 切換模型到評估模式
    model.eval()


In [ ]:
# 得到了bzq 正確的函數，拿來做imagenet 正確的預測

def single_data_bzq_mask_preprocessing_imagenet(original_data, start_x, start_y, len_x, len_y, magnification):
    if len_x <= 0 or len_y <= 0:
        return original_data
    new_data = np.copy(original_data.cpu())
    new_data[:, start_y:start_y + len_y, start_x:start_x + len_x] *= magnification
    #new_data[start_y:start_y + len_y, start_x:start_x + len_x, :] *= magnification
    return new_data


#print(random_num_for_bzq_mask_imagenet)

def single_data_bzq_mask_preprocessing_imagenet_random_global(original_data, start_x, start_y, len_x, len_y, random_num_for_bzq_mask_imagenet):
    if len_x <= 0 or len_y <= 0:
        return original_data
    new_data = np.copy(original_data.cpu().numpy())
    new_data = new_data.squeeze(0)
    random_num_for_bzq_mask_imagenet = random_num_for_bzq_mask_imagenet[:, :len_y, :len_x] 
    new_data[:, start_y:start_y + len_y, start_x:start_x + len_x] = random_num_for_bzq_mask_imagenet
    return new_data


bzq = []
correct_predictions_imagenet = []
incorrect_predictions_imagenet = []
bzq_imagenet = []

#bzq = 0的時候，提取mask之下的softmax
correct_predictions_bzq_zero_softmax_mean = []
correct_predictions_bzq_zero_softmax_std = []
incorrect_predictions_bzq_zero_softmax_mean = []
incorrect_predictions_bzq_zero_softmax_std = []


In [ ]:
class DummyDataset(Dataset):
    def __init__(self, limit):
        self.limit = limit

    def __len__(self):
        return self.limit

    def __getitem__(self, idx):
        img = torch.randn(3, 224, 224)
        label = torch.tensor(-1)
        return img, label

test_dataset = DummyDataset(limit)

# 創建 DataLoader
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=6, pin_memory=True)

for img, lab in test_loader:
    print(img.shape, lab.shape)
    break

In [ ]:
conf_vanilla = []
# 預測並顯示分數

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device).float()
        labels = labels.to(device)
        outputs = model(images)

        # 計算Softmax並獲取最大值
        softmax_probs = F.softmax(outputs, dim=1)  # Softmax值
        max_probs, predicted = torch.max(softmax_probs, 1)

        conf_vanilla.extend(max_probs.cpu().numpy()) 
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #print(model)
        #print(labels, predicted)
        
conf_vanilla = np.array(conf_vanilla)
accuracy = correct / total
print(f'Accuracy: {accuracy * 100:.2f}%')

In [ ]:
# bzq modifying
len_x = len_x_target
len_y = len_y_target
stride_x = stride_x_target
stride_y = stride_y_target
batch_size = 1  # 設定批次大小

original_predictions_imagenet = []

with torch.no_grad():
    for batch_data, batch_labels in test_loader:
        batch_data = batch_data.to(device).float()
        batch_predictions_imagenet = model(batch_data).cpu().numpy()
        original_predictions_imagenet.append(batch_predictions_imagenet)

original_predictions_imagenet = np.vstack(original_predictions_imagenet)

# 使用正確的方式來訪問 test_dataset 中的標籤
test_labels = [test_dataset[i][1] for i in range(len(test_dataset))]

predicted_labels = np.argmax(original_predictions_imagenet, axis=1)
correct_indices = np.where(predicted_labels == np.array(test_labels))[0]
incorrect_indices = np.where(predicted_labels != np.array(test_labels))[0]

correct_predictions_imagenet.extend(correct_indices.tolist())
incorrect_predictions_imagenet.extend(incorrect_indices.tolist())

print(f"{len(correct_predictions_imagenet)}, {len(incorrect_predictions_imagenet)}")


In [ ]:
import torch
import numpy as np
from collections import Counter
import torch.nn.functional as F

random_num_for_bzq_mask_imagenet = np.random.randint(0, 256, (3, len_y_target, len_x_target)).astype(np.float32) / 255.0

def compute_entropy(softmax_predictions):
    # 為了數值穩定，加上 ε 避免 log(0)
    eps = 1e-8
    softmax_predictions = torch.mean(softmax_predictions, dim=0)
    log_probs = torch.log(softmax_predictions + eps)
    entropy = -torch.sum(softmax_predictions * log_probs)
    return entropy

def process_batch(batch_data, batch_labels, len_y, stride_y, len_x, stride_x, device, model, alpha, bzq):
    if len(batch_data) == 0:
        return bzq, np.array([])

    targets = []

    for i in range(0, 224 - len_y, stride_y):
        for j in range(0, 224 - len_x, stride_x):
            #target = single_data_bzq_mask_preprocessing_imagenet(batch_data, i, j, len_x, len_y, 0)
            target = single_data_bzq_mask_preprocessing_imagenet_random_global(batch_data, i, j, len_x, len_y, random_num_for_bzq_mask_imagenet)
            targets.append(target)
    
    targets_tensor = torch.from_numpy(np.vstack(targets).reshape(-1, 3, 224, 224)).float().to(device)
    predictions = model(targets_tensor)
    
    #temp, _ = torch.max(F.softmax(predictions), dim=1)
    #temp = temp.cpu().numpy()
    
    max_bzq_indices = torch.argmax(predictions, dim=1).cpu().numpy()
    
    # 計算 softmax
    softmax_predictions = F.softmax(predictions, dim=1)

    #print(softmax_predictions.shape)

    entropy = compute_entropy(softmax_predictions)

    labels = batch_labels

    counter = Counter(max_bzq_indices)
    most_common_num, most_common_count = counter.most_common(1)[0]
    
    temp = softmax_predictions[:, most_common_num].cpu().numpy()
    
    bzq.append(alpha * np.mean(temp) + (1 - alpha) * (2.0 / np.pi * np.arctan(1.0 / np.std(temp))))

    return bzq, temp, entropy.cpu()

entropies = []

with torch.no_grad():
    for batch_data, batch_labels in test_loader:
        batch_data = batch_data.to(device)  # 將批次影像搬到指定裝置
        batch_labels = batch_labels.to(device)
        if len(batch_data) > 0:
            bzq, temp, entropy = process_batch(
                batch_data, batch_labels, len_y, stride_y, len_x, stride_x, device, model, alpha, bzq
            )
            entropies.append(entropy)
            # 影像預處理
            original_data = single_data_bzq_mask_preprocessing_imagenet(batch_data, 0, 0, 0, 0, 0)
            original_prediction = model(torch.tensor(original_data.reshape(-1, 3, 224, 224)).float().to(device))
            max_original_index = torch.argmax(original_prediction).item()

            # 根據 `bzq` 值分類統計
            if bzq[-1] == 0.0:
                if (len(bzq) - 1) in correct_predictions_imagenet:
                    correct_predictions_bzq_zero_softmax_mean.append(np.mean(temp))
                    correct_predictions_bzq_zero_softmax_std.append(np.std(temp))
                else:
                    incorrect_predictions_bzq_zero_softmax_mean.append(np.mean(temp))
                    incorrect_predictions_bzq_zero_softmax_std.append(np.std(temp))

bzq_imagenet = np.array(bzq)



In [ ]:
print(entropies)


In [ ]:

# 計算熵的分佈（分桶）
bins = np.linspace(min(entropies), max(entropies), 15) 
hist, bin_edges = np.histogram(entropies, bins=bins)  # 計算直方圖數據
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2  # 計算每個 bin 的中心點
print(np.median(entropies), np.std(entropies))
# 繪製折線圖
plt.figure(figsize=(10, 5))
plt.plot(bin_centers, hist, linestyle='-', marker='o')

# 設定標題與軸標籤
plt.title("Entropy Distribution")
plt.xlabel("Entropy")
plt.ylabel("# of Examples")
plt.legend()
plt.grid(True)
plt.show()